In [1]:
using StatisticalRethinking, CmdStan, StanMCMCChain
gr(size=(500,500));

ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

df2 = filter(row -> row[:age] >= 18, df)
first(df2, 5)

heightsmodel = "
// Inferring a Rate
data {
  int N;
  real<lower=0> h[N];
}
parameters {
  real<lower=0> sigma;
  real<lower=0,upper=250> mu;
}
model {
  // Priors for mu and sigma
  mu ~ normal(178, 20);
  sigma ~ uniform( 0 , 50 );

  // Observed heights
  h ~ normal(mu, sigma);
}
";

stanmodel = Stanmodel(name="heights", monitors = ["mu", "sigma"],model=heightsmodel,
  output_format=:mcmcchain);

heightsdata = Dict("N" => length(df2[:height]), "h" => df2[:height]);

rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  CmdStanDir=CMDSTAN_HOME);

describe(chn)

serialize("m4.1s.jls", chn)
chn2 = deserialize("m4.1s.jls")

describe(chn2)


File /Users/rob/.julia/dev/StatisticalRethinking/scripts/04/tmp/heights.stan will be updated.


Inference for Stan model: heights_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (0.035, 0.034, 0.035, 0.037) seconds, 0.14 seconds total
Sampling took (0.070, 0.044, 0.086, 0.046) seconds, 0.25 seconds total

                Mean     MCSE  StdDev    5%   50%   95%    N_Eff  N_Eff/s    R_hat
lp__            -892  2.4e-02     1.0  -893  -891  -891  1.9e+03  7.6e+03  1.0e+00
accept_stat__   0.92  1.0e-02   0.099  0.72  0.96   1.0  9.5e+01  3.9e+02  1.0e+00
stepsize__      0.78  7.2e-02    0.10  0.64  0.78  0.93  2.0e+00  8.1e+00  1.1e+14
treedepth__      2.2  1.6e-01    0.99   1.0   2.0   4.0  3.6e+01  1.5e+02  1.0e+00
n_leapfrog__     7.7  2.2e+00      16   1.0   3.0    27  5.1e+01  2.1e+02  1.0e+00
divergent__     0.00      nan    0.00  0.00  0.00  0.00      nan      nan      nan
energy__         893  3.4e-02     1

end of m4.1s#-
*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*